In [17]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import time


In [18]:
# Load MNIST data
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Preprocess data
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Convert grayscale to RGB by repeating the single channel
x_train = tf.image.grayscale_to_rgb(tf.expand_dims(x_train, -1))
x_test = tf.image.grayscale_to_rgb(tf.expand_dims(x_test, -1))

# Resize images to 32x32
x_train = tf.image.resize(x_train, [32, 32])
x_test = tf.image.resize(x_test, [32, 32])

# Convert labels to categorical
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)


In [19]:
def build_resnet50_gpu():
    # Use ResNet50 from Keras applications
    base_model = ResNet50(weights=None, include_top=False, input_shape=(32, 32, 3))

    # Add custom top layers
    x = base_model.output
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dense(10, activation='softmax')(x)

    model = tf.keras.Model(inputs=base_model.input, outputs=x)

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model


In [ ]:
# Train and evaluate the ResNet-50 model on CPU
with tf.device('/CPU:0'):
    model_cpu = build_resnet50_gpu()
    print("Training ResNet-50 Model on CPU.......")

    # Record training time
    start_time = time.time()
    history_gpu = model_cpu.fit(x_train, y_train, epochs=1, batch_size=64, validation_split=0.1, verbose=2)
    end_time = time.time()
    cpu_training_time = end_time - start_time

    # Evaluate model
    start_time = time.time()
    test_loss, test_acc = modemodel_cpul_gpu.evaluate(x_test, y_test, verbose=2)
    end_time = time.time()
    cpu_inference_time = end_time - start_time

print(f"CPU Training Time: {cpu_training_time:.2f} seconds")
print(f"CPU Inference Time: {cpu_inference_time:.2f} seconds")


Training ResNet-50 Model on CPU.......


In [ ]:
class CustomConv2D(tf.keras.layers.Layer):
    def __init__(self, filters, kernel_size, strides=(1, 1), padding='SAME', **kwargs):
        super(CustomConv2D, self).__init__(**kwargs)
        self.filters = filters
        self.kernel_size = kernel_size
        self.strides = strides
        self.padding = padding

    def build(self, input_shape):
        self.kernel = self.add_weight(
            shape=(*self.kernel_size, input_shape[-1], self.filters),
            initializer='glorot_uniform',
            trainable=True
        )

    def call(self, inputs):
        return tf.nn.conv2d(
            inputs,
            self.kernel,
            strides=[1, *self.strides, 1],
            padding=self.padding
        )


In [ ]:
def residual_block(x, filters, kernel_size=(3, 3), strides=(1, 1), padding='SAME'):
    shortcut = x
    x = CustomConv2D(filters, kernel_size, strides=strides, padding=padding)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.ReLU()(x)

    x = CustomConv2D(filters, kernel_size, strides=strides, padding=padding)(x)
    x = tf.keras.layers.BatchNormalization()(x)

    if x.shape[-1] != shortcut.shape[-1]:
        shortcut = CustomConv2D(filters, (1, 1), strides=strides, padding=padding)(shortcut)
        shortcut = tf.keras.layers.BatchNormalization()(shortcut)

    x = tf.keras.layers.add([x, shortcut])
    x = tf.keras.layers.ReLU()(x)
    return x


In [ ]:
def build_custom_resnet50():
    inputs = tf.keras.Input(shape=(32, 32, 3))

    # Initial Conv Layer
    x = CustomConv2D(64, (7, 7), strides=(2, 2))(inputs)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.ReLU()(x)
    x = tf.keras.layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='SAME')(x)

    # Residual Blocks
    x = residual_block(x, 64)
    x = residual_block(x, 64)
    x = residual_block(x, 64)

    x = residual_block(x, 128, strides=(2, 2))
    x = residual_block(x, 128)
    x = residual_block(x, 128)

    x = residual_block(x, 256, strides=(2, 2))
    x = residual_block(x, 256)
    x = residual_block(x, 256)

    x = residual_block(x, 512, strides=(2, 2))
    x = residual_block(x, 512)

    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dense(10, activation='softmax')(x)

    model = tf.keras.Model(inputs, x)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model


In [ ]:
# Train and evaluate the custom ResNet-50 model on GPU
with tf.device('/GPU:0'):
    model_gpu = build_custom_resnet50()
    print("Training Custom ResNet-50 Model on GPU...")

    start_time = time.time()
    history_cpu = model_gpu.fit(x_train, y_train, epochs=1, batch_size=64, validation_split=0.1, verbose=2)
    end_time = time.time()
    gpu_training_time = end_time - start_time

    # Evaluate model
    start_time = time.time()
    test_loss, test_acc = model_gpu.evaluate(x_test, y_test, verbose=2)
    end_time = time.time()
    gpu_inference_time = end_time - start_time

print(f"GPU Training Time: {gpu_training_time:.2f} seconds")
print(f"GPU Inference Time: {gpu_inference_time:.2f} seconds")
